# **Fingerprints Finger Classification**
## JCE - Final Project 
### By Kobi Amsellem & Zohar Kedem

#### Find a model that helps a forensic investigator decide To which finger does the fingerprint belong
#### 10 classeses Classification (thumnb, index, middle, ring, little)*(right, left)

Expirements:
1. decide of using transfer learning or full training 
2. Comparison between couple of backbones models 
3. Comparison optimizers
4. Comparison learning rates
5. Comparison loss function

* train best model for deployment

In [ ]:
from datasets import SOCOFingFingers
import FPMLmodule.backbones as backbones
import FPMLmodule.classifiers as classifiers
import FPMLmodule.utils as utils
from FPMLmodule.fpml import FPML 
from tensorflow.keras.optimizers import Adam, Nadam, RMSprop
import copy

import tensorflow as tf
print("Tensorflow version " + tf.__version__)

### **Hyperparameters and default settings**  ###

* define the global configuration for training
* define datset configuration as split, batch size etc.
* declare relevant backbones weights path
* set epochs for study and final model


In [ ]:
# Global Config
seed=9
imgDim = (120, 120, 3)
imgHeight, imgWidth, imgChannels = imgDim
batchSize = 32

# Dataset configuration
dsConfig = {
    'batchSize': batchSize, 
    'parallelTune': tf.data.AUTOTUNE, 
    'split': [0.7, 0.15, 0.15], 
    'inputDim': imgDim, 
    'seed': seed, 
    'shuffle': True
    }

weightsRN50 = "./weights/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5"
weightsMNV2 = "./weights/mobilenet_v2_weights_tf_dim_ordering_tf_kernels_1.0_224_no_top.h5"
weightsENB2 = "./weights/efficientnetb2_notop.h5"
weightsINCEPTIONV3 = "./weights/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5"
weightsXCEPTION = "./weights/xception_weights_tf_dim_ordering_tf_kernels_notop.h5"

# Training interval
epochsForSearch = 10
epochsForBest = 100

### **Create Dataset**

* labeld as Male/Female 
* sampling mode
* split to train/test/validation sub-datasets

In [ ]:
SOCOFingers = SOCOFingFingers(**dsConfig)
fingersDs = SOCOFingers.create()
trainDs, testDs, valDs = fingersDs

### Show Datasets Split Sizes and split diversity ###

In [ ]:
utils.displayDatasetSplitInformation(fingersDs, SOCOFingers.classNames)
utils.displayDsSamples(trainDs, shape=(5,5), classNames=SOCOFingers.classNames)

### Define base hyperparameters for study ###
* optimizer
* learning rate
* loss function
* accuracy metric

In [ ]:
nbClasses = len(SOCOFingers.classNames)
activation = "softmax"
hypers = {
    "optimizer": Adam,
    "learningRate": 0.001,
    "loss": 'binary_crossentropy',
    "metrics": 'accuracy'
}

### **Using MobileNetV2 to compare transfer learning options**

define 3 studies using mobilenet as backbone to compare between:
* transfer learning from imagenet and train the classifier only
* transfer learning from imagenet and train the whole model
* train the model from scratch

In [ ]:


trainingModeStudy = {
    "mobilenetV2-pretrained-untrainable" : {
        "architecture": {
            "backbone": backbones.MobileNetV2(imgDim, weights=weightsMNV2, trainable=False),
            "classfier": classifiers.DefaultClassifier(nbClasses, activation),
            "inputLayer": None, 
            "inputDim": imgDim
        },
        "hyperparameters":hypers
    },
    "MobileNetV2-pretrained-trainable" : {
        "architecture": {
            "backbone": backbones.MobileNetV2(imgDim, weights=weightsMNV2, trainable=True),
            "classfier": classifiers.DefaultClassifier(nbClasses, activation),
            "inputLayer": None, 
            "inputDim": imgDim
        },
        "hyperparameters":hypers
    },
    "MobileNetV2-fulltraining" : {
        "architecture": {
            "backbone": backbones.MobileNetV2(imgDim, weights=None, trainable=True),
            "classfier": classifiers.DefaultClassifier(nbClasses, activation),
            "inputLayer": None, 
            "inputDim": imgDim
        },
        "hyperparameters":hypers
    }
}

trainingModeHistories = utils.researchStudies(trainDs, valDs, trainingModeStudy, epochsForSearch)


#### examine transfer learing mode results 

In [ ]:
utils.displayStudiesProgress(trainingModeHistories)
trainingModeBestModel = utils.getBestStudyFromHistories(trainingModeHistories)
pretrained = 'pretrained' in trainingModeBestModel
trainable = 'untrainable' not in trainingModeBestModel
print(trainingModeBestModel)

### **Comparing Backbones**

define studies using different backbone to compare between:
* ResNet50
* EfficientNetB2
* InceptionV3
* Xception
* MobileNetV2


In [ ]:
bestBackboneStudy = {
    "resnet50" : {
        "architecture": {
            "backbone": backbones.ResNet50(imgDim, weights=weightsRN50, trainable=trainable),
            "classfier": classifiers.DefaultClassifier(nbClasses, activation),
            "inputLayer":"", 
            "inputDim": imgDim
        },
        "hyperparameters": hypers
    },
    "efficientnet" : {
        "architecture": {
            "backbone": backbones.EfficientNetB2(imgDim, weights=weightsENB2, trainable=trainable),
            "classfier": classifiers.DefaultClassifier(nbClasses, activation),
            "inputLayer":"", 
            "inputDim": imgDim
        },
        "hyperparameters": hypers
    },
    "inceptionV3" : {
        "architecture": {
            "backbone": backbones.InceptionV3(imgDim, weights=weightsINCEPTIONV3, trainable=trainable),
            "classfier": classifiers.DefaultClassifier(nbClasses, activation),
            "inputLayer":"", 
            "inputDim": imgDim
        },
        "hyperparameters": hypers
    },
    "xception" : {
        "architecture": {
            "backbone": backbones.Xception(imgDim, weights=weightsXCEPTION, trainable=trainable),
            "classfier": classifiers.DefaultClassifier(nbClasses, activation),
            "inputLayer":"", 
            "inputDim": imgDim
        },
        "hyperparameters": hypers
    },
    
}

bestBackboneHistories = utils.researchStudies(trainDs, valDs, bestBackboneStudy, epochsForSearch)

#### examine backbone study results 

In [ ]:
bestBackboneHistories[trainingModeBestModel] = trainingModeHistories[trainingModeBestModel]
bestBackboneStudy[trainingModeBestModel] = trainingModeStudy[trainingModeBestModel]
utils.displayStudiesProgress(bestBackboneHistories)
bestBackbone = utils.getBestStudyFromHistories(bestBackboneHistories)
print(bestBackbone)

### **Comparing Optimizers**

define studies using different optimizers to compare between:
* Nadam
* RMSprop
* Adam


In [ ]:
optimizers = {
    'Nadam': Nadam,
    'RMSprop': RMSprop
    }

optimizersStudy = dict()

for opt in optimizers:
    studyName = bestBackbone+"_"+opt
    optimizersStudy[studyName] = copy.deepcopy(bestBackboneStudy[bestBackbone])
    optimizersStudy[studyName]["hyperparameters"]["optimizer"] = optimizers[opt]

optimizersHistories = utils.researchStudies(trainDs, valDs, optimizersStudy, epochsForSearch)

In [ ]:
optimizersHistories[bestBackbone+"_Adam"] = bestBackboneHistories[bestBackbone]
optimizersStudy[bestBackbone+"_Adam"] = bestBackboneStudy[bestBackbone]
utils.displayStudiesProgress(optimizersHistories)
bestOptimizier = utils.getBestStudyFromHistories(optimizersHistories)
print(bestOptimizier)

### **Comparing Learning rates**

define studies using different lr to compare between:
* 0.1
* 0.01
* 0.001
* 0.0001
* 0.00001


In [ ]:
learningRates = [0.1, 0.1e-1, 0.1e-3, 0.1e-4]

learningRatesStudy = dict()

for lr in learningRates:
    studyName = bestOptimizier+"_"+str(lr)
    learningRatesStudy[studyName] = copy.deepcopy(optimizersStudy[bestOptimizier])
    learningRatesStudy[studyName]["hyperparameters"]["learningRate"] = lr

    
learningRatesHistories = utils.researchStudies(trainDs, valDs, learningRatesStudy, epochsForSearch)

In [ ]:
learningRatesHistories[bestOptimizier+"_"+str(hypers["learningRate"])] = optimizersHistories[bestOptimizier]
learningRatesStudy[bestOptimizier+"_"+str(hypers["learningRate"])] = optimizersStudy[bestOptimizier]
utils.displayStudiesProgress(learningRatesHistories)
bestLearningRate = utils.getBestStudyFromHistories(learningRatesHistories)
print(bestLearningRate)

### **Comparing Loss Functions**

define studies using different loss function to compare between:
* binary-focal-crossentropy
* hinge
* binary-crossentropy


In [ ]:
losses = ['binary_focal_crossentropy', 'hinge']

lossStudy = dict()
for ls in losses:
    studyName = bestLearningRate+"_"+str(ls)
    lossStudy[studyName] = copy.deepcopy(learningRatesStudy[bestLearningRate])
    lossStudy[studyName]["hyperparameters"]["loss"] = ls

lossesHistories = utils.researchStudies(trainDs, valDs, lossStudy, epochsForSearch)

In [ ]:
lossesHistories[bestLearningRate+"_"+str(hypers["loss"])] = learningRatesHistories[bestLearningRate]
lossStudy[bestLearningRate+"_"+str(hypers["loss"])] = learningRatesStudy[bestLearningRate]
utils.displayStudiesProgress(lossesHistories)
bestLoss = utils.getBestStudyFromHistories(lossesHistories)
print(bestLoss)

In [ ]:
bestModelName = bestLoss
bestModelSetting = lossStudy[bestLoss]


In [ ]:
trainValDs = trainDs.concatenate(valDs)
bestModel = FPML(**bestModelSetting["architecture"]).create(**bestModelSetting["hyperparameters"])
bestModelHistory = bestModel.fit(trainValDs, validation_data=testDs, epochs=epochsForBest, verbose=1)

In [ ]:
utils.displayStudiesProgress({"BestModel\n"+bestModelName : bestModelHistory})

In [ ]:
utils.displayConfusion(testDs, bestModel)

In [ ]:
bestModel.save('./weights/'+bestModelName+'_SOCOFingers.h5')